In [1]:
import numpy as np
import itertools

In [2]:
filename='./D10input.txt'
with open(filename,'r') as fh: # read in files
    temp = fh.read().splitlines()

# Part1 #

In [3]:
# produce map of astroids coordinates
arr=[]
for s in temp:
    arr.append(list(s)) # split each row into characters
mp=np.array(arr) # astroid map
dim=mp.shape # dimension of the map
asts=np.argwhere(mp=='#') #astroids coordinates

In [4]:
def gcd(a,b):  # greatest common divisor
    if(b==0): 
        return a 
    else: 
        return gcd(b,a%b) 

In [5]:
# all direction vectors with coprime components
dircs=np.array(list(itertools.product(range(1,dim[0]),range(1,dim[1])))) # find all possible directions in the 1st quadrant
gcds=np.array([gcd(*c) for c in dircs])
cp_dircs=dircs[gcds==1] # direction vector with coprime x and y compnents 
cp_dircs2=cp_dircs.copy() 
cp_dircs2[:,1]=-cp_dircs[:,1] # direction vector in the 4th quadrant
cp_dircs=np.append(cp_dircs,cp_dircs2,axis=0)
cp_dircs=np.append(cp_dircs,[[0,1],[1,0]],axis=0) # add direction vectors of x and y direction

In [6]:
# number of detectable astroids for each astroid location
n_detect=np.zeros_like(asts[:,0],dtype=int)
for dirc in cp_dircs: # choose one directoin vector
    # evaluate z=ax+by: inner product between the direction vector(a,b) and the coordinate of the astroid(x,y)
    values=dirc[0]*asts[:,0]+dirc[1]*asts[:,1]
    uni_values,counts=np.unique(values,return_counts=True) # find the unique 'z's and how many times they appeared
    # mask: if 2 or more points get the same value z means they are in each others LOS
    for uval in uni_values[counts>1]:
        msk1=(values==uval) # astroids on the line
        # find astroids on the ends of the line, only '+1' to n_detect
        if ((dirc[0]!=1)&(dirc[1]!=0)): # line not perpendicular to x axis
            x_coords=asts[:,0][msk1] # x coordinates of the astroids on the line
            x_min=np.amin(x_coords)
            x_max=np.amax(x_coords)
            msk2=(asts[:,0]==x_min)|(asts[:,0]==x_max) # astroids on the ends of the line
        # line perpendicular to x axis, use y coordinates to find the end points
        else: 
            y_coords=asts[:,1][msk1]
            y_min=np.amin(y_coords)
            y_max=np.amax(y_coords)
            msk2=(asts[:,1]==y_min)|(asts[:,1]==y_max)

        # apply mask to the map of number counts
        n_detect[msk1&(~msk2)]+=2 # astroids in the middle of the line can detect 2 other astroids
        n_detect[msk1&msk2]+=1 # astroids on the ends can only detect 1 other astroid

In [7]:
np.amax(n_detect)

282

# Part2 #

In [8]:
station=asts[n_detect.argmax()] # coordinates of the station

In [9]:
recds=asts-station #coordinates relative to the station
recds=np.delete(recds,np.arange(len(recds))[(recds[:,0]==0)&(recds[:,1]==0)],axis=0) # remove (0,0), the station itself

In [10]:
theta=np.arctan2(recds[:,1], -recds[:,0]) # angle theta respect to y axis and rotating clockwise
theta=np.where((-np.pi<theta)&(theta<0),theta+2*np.pi,theta) # fix angle: [-pi, pi]->[0, 2pi]
r=np.hypot(recds[:,1], -recds[:,0]) # distance to the station

In [11]:
# create a ndarray with columns: theta, r, y, x of the astroids in the relative coordinates
all_data=np.rec.fromarrays([theta, r, recds[:,0],recds[:,1]], dtype=[('theta','float'),('r','float'),('y','int'),('x','int')])
all_data.sort(order=['theta','r']) # first sort in angle then in distance

In [12]:
data_copy=all_data.copy()
corder=np.empty((2,0),dtype=int) # initialize: coordinates of the astroids evaporated in order
while len(data_copy)>0:
    # for astroids with the same angles, only the astroid with smallest distance to the station gets evaporated in one rotation
    _, inds=np.unique(data_copy['theta'],return_index=True) 
    corder=np.hstack((corder,np.array([data_copy['y'][inds],data_copy['x'][inds]]))) # append the coordinates of the evaporated
    data_copy=np.delete(data_copy,inds) # remove the evaporated astroids from the map
corder=corder.T+station # restore absolute coordinates

In [13]:
corder[200-1]

array([ 8, 10])